In [172]:
import os
import time
from sklearn.tree import DecisionTreeClassifier
from numpy import array
import pandas as pd
import numpy as np
from numpy import array
from numpy import zeros
os.chdir("D:\\practicespace\\github\\Help_Navigate_Robots\\dataset")

### 将数据进行join操作，然后按照分类进行排序（用于后面统计各个分类数量）

In [199]:
def getSampleData():
    # 读入数据，加载到Pandas里面
    start_time = time.time()
    pd_x = pd.read_csv("x_train.csv")
    end_time = time.time()
    #print("pandas阅读csv文件耗时: ", end_time - start_time)
    pd_y = pd.read_csv("y_train.csv")
    pd_data = pd_x.set_index("series_id").join(pd_y.set_index("series_id"))
    
    # 对数据列进行归一化
    max_min_scalar = lambda x: (x - np.min(x))/(np.max(x) - np.min(x))
    normed_Z = pd_data[["linear_acceleration_Z"]].apply(max_min_scalar) # 注意这里一定是套了两层中括号
    normed_Y = pd_data[["linear_acceleration_Y"]].apply(max_min_scalar)
    pd_data["linear_acceleration_Z"] = normed_Z
    pd_data["linear_acceleration_Y"] = normed_Y
    
#     # 下面对于相关性进行计算，删掉相关性小的字段
#     pd_data.drop("linear_acceleration_X", axis=1, inplace=True)
#     pd_data.drop("angular_velocity_X", axis=1, inplace=True)
#     pd_data.drop("linear_acceleration_Z", axis=1, inplace=True)    
#     print("删除相关性低的列之后...")
#     print(pd_data.head(3))
    
    pd_data_sorted = pd_data.sort_values(by=["surface"])
    sample_train = pd_data_sorted.values[:, 2:]
    #print(sample_train[:3, :])
    totalCount = shape(sample_train)[0]
    print("样本总数为: %s, 0.6分割点: %s" % (totalCount, int(totalCount * 0.6)))
    
    return sample_train

getSampleData()

样本总数为: 487680, 0.6分割点: 292608


array([[-0.65046, 0.74478, 0.10758, ..., 0.4686493184634448, 7, 'carpet'],
       [-0.9636899999999999, 0.22115, 0.023597, ..., 0.4573623650203576,
        7, 'carpet'],
       [-0.9635, 0.22171999999999997, 0.023839, ..., 0.45401309966365727,
        7, 'carpet'],
       ...,
       [0.67332, -0.72425, -0.10549000000000001, ...,
        0.49297008320056646, 38, 'wood'],
       [0.67156, -0.72587, -0.10582000000000001, ..., 0.4647980173482032,
        38, 'wood'],
       [-0.089847, -0.98464, -0.14944000000000002, ...,
        0.4669647725261108, 43, 'wood']], dtype=object)

In [200]:
# # 读入数据，加载到Pandas里面
# start_time = time.time()
# pd_x = pd.read_csv("x_train.csv")
# end_time = time.time()
# #print("pandas阅读csv文件耗时: ", end_time - start_time)
# pd_y = pd.read_csv("y_train.csv")
# pd_data = pd_x.set_index("series_id").join(pd_y.set_index("series_id"))
# print(pd_data.head(3))
# # 分析一下相关性，但是发现除了“measurement_number，linear_acceleration_X，angular_velocity_X，linear_acceleration_Z”
# # 无关之外其他字段还好
# corr_mastrix = pd_data.corr()
# corr_mastrix
# corr_mastrix["group_id"].sort_values(ascending=False)

In [201]:
# 统计各个分类的数量，用于后续划分测试集合和验证集合
def getClassCount(sample_train):
    classCount = {}

    for sample in sample_train:
        className = sample[-1]
        classCount[className] = classCount.get(className, 0) + 1

    return classCount

sample_train = getSampleData()
getClassCount(sample_train)

样本总数为: 487680, 0.6分割点: 292608


{'carpet': 24192,
 'concrete': 99712,
 'fine_concrete': 46464,
 'hard_tiles': 2688,
 'hard_tiles_large_space': 39424,
 'soft_pvc': 93696,
 'soft_tiles': 38016,
 'tiled': 65792,
 'wood': 77696}

In [202]:
from numpy import array

def splitTrainAndValidateData(sample_train, classCount):
    # 获取训练集合以及验证集合
    data_x = []
    data_y = []
    data_x_train = []
    data_y_train = []
    data_x_validate = []
    data_y_validate = []

    target = ""
    seria_count = 0
    validate_sample_count = 100

    for sample in sample_train:
        #print(sample)
        x = sample[:-1]
        data_x.append(x)
        y = sample[-1]
        data_y.append(y)

        if y != target:
            target = y
            seria_count = 0
            validate_sample_count = int(classCount[target]*0.6)
            #print("class: [%s] validate count: %d " % ( target, validate_sample_count))
        seria_count += 1
        if(seria_count <= validate_sample_count):
            #print("target is: %s, seria_count is: %s, branch: train" % (target, seria_count))
            data_x_train.append(x)
            data_y_train.append(y)
        else:
            #print("target is: %s, seria_count is: %s, branch: validate" % (target, seria_count))
            data_x_validate.append(x)
            data_y_validate.append(y)

    data_x_train_array = array(data_x_train)
    data_y_train_array = array(data_y_train)
    data_x_validate_array = array(data_x_validate)
    data_y_validate_array = array(data_y_validate)

    print("data_x_train_array shape: ", shape(data_x_train_array))
    print("data_y_train_array shape: ", shape(data_y_train_array))
    print("data_x_validate_array shape: ", shape(data_x_validate_array))
    print("data_y_validate_array shape: ", shape(data_y_validate_array))
    
    return data_x_train_array, data_y_train_array, data_x_validate_array, data_y_validate_array

sample_train = getSampleData()
classCount = getClassCount(sample_train)
data_x_train_array, data_y_train_array, data_x_validate_array, data_y_validate_array = splitTrainAndValidateData(sample_train, classCount)

样本总数为: 487680, 0.6分割点: 292608
data_x_train_array shape:  (292604, 11)
data_y_train_array shape:  (292604,)
data_x_validate_array shape:  (195076, 11)
data_y_validate_array shape:  (195076,)


In [212]:
def fitAndValidate(ctl, data_x_train_array, data_y_train_array, data_x_validate_array, data_y_validate_array):
    # 训练
    ctl.fit(data_x_train_array, data_y_train_array)
    # 预测
    y_predict = ctl.predict(data_x_validate_array)
    size = len(y_predict)
    # 统计错误率
    ok_count = 0
    error_count  = 0
    for index in range(size):
        if y_predict[index] == data_y_validate_array[index]:
            ok_count +=1
        else:
            error_count += 1
    # 输出结果
    print("数据总量%d，错误数：%d，错误率：%f" %(size, error_count, error_count/size))
    scores = cross_val_score(knn, data_x_train_array, data_y_train_array, cv=5, scoring="accuracy")
    print("cross_val_score: ", scores)

In [204]:
# 获取训练数据集
def getData():
    sample_train = getSampleData()
    classCount = getClassCount(sample_train)
    return splitTrainAndValidateData(sample_train, classCount)

data_x_train_array, data_y_train_array, data_x_validate_array, data_y_validate_array = getData()

样本总数为: 487680, 0.6分割点: 292608
data_x_train_array shape:  (292604, 11)
data_y_train_array shape:  (292604,)
data_x_validate_array shape:  (195076, 11)
data_y_validate_array shape:  (195076,)


### knn

In [209]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
fitAndValidate(knn, data_x_train_array, data_y_train_array, data_x_validate_array, data_y_validate_array)

from sklearn.model_selection import cross_val_score
cross_val_score(knn, data_x_train_array, data_y_train_array, cv=3, scoring="accuracy")
# 未降维之前
# 数据总量195076，错误数：13042，错误率：0.066856

# 去掉了measurement_number列之后，
# 数据总量195076，错误数：1238，错误率：0.006346

# 对于“linear_acceleration_Y”和“linear_acceleration_Z”归一化之后
# 数据总量195076，错误数：80，错误率：0.000410

# 降维后（删掉了不相关的列之后）
# 数据总量195076，错误数：0，错误率：0.000000

数据总量195076，错误数：80，错误率：0.000410


array([0.97432795, 0.99888244, 0.99947709])

### Decision Tree

In [211]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth=12, random_state=42)
fitAndValidate(tree, data_x_train_array, data_y_train_array, data_x_validate_array, data_y_validate_array)

cross_val_score(knn, data_x_train_array, data_y_train_array, cv=5, scoring="accuracy")

# 未降维之前
# 数据总量195076，错误数：2527，错误率：0.012954

# 去掉了measurement_number列之后，
# 数据总量195076，错误数：1928，错误率：0.009883

# 对于“linear_acceleration_Y”和“linear_acceleration_Z”归一化之后
# 数据总量195076，错误数：1928，错误率：0.009883

# 降维后（删掉了不相关的列之后）
# 数据总量195076，错误数：2768，错误率：0.014189

数据总量195076，错误数：1928，错误率：0.009883


array([0.99690725, 0.99974369, 0.99948735, 0.99931646, 0.99947024])

### RandomForest

In [213]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators=300,max_depth=15, max_leaf_nodes=15, n_jobs=-1)
fitAndValidate(tree, data_x_train_array, data_y_train_array, data_x_validate_array, data_y_validate_array)
# 未降维前
# 数据总量195076，错误数：2620，错误率：0.013431

# 去掉了measurement_number列之后，
# 数据总量195076，错误数：1928，错误率：0.009883

# 对于“linear_acceleration_Y”和“linear_acceleration_Z”归一化之后
# 数据总量195076，错误数：1928，错误率：0.009883

# 降维后（删掉了不相关的列之后）
# 数据总量195076，错误数：2768，错误率：0.014189

数据总量195076，错误数：1928，错误率：0.009883
cross_val_score:  [0.99690725 0.99974369 0.99948735 0.99931646 0.99947024]


### SGDClassifier

In [208]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(random_state=42)
fitAndValidate(sgd, data_x_train_array, data_y_train_array, data_x_validate_array, data_y_validate_array)
# 错误率70%

d:\software\develop\python\python35\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


数据总量195076，错误数：148914，错误率：0.763364


### 总结
### knn对于归一化还是比较敏感，但是下面的tree和randomForest则发现对于归一化没有反应；但是大部分的算法对于measurement_number这一列去掉之后的反应还是很明显的，这一列就是一个自增长，和分类毫无关联